# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f54b05f25b0>
├── 'a' --> tensor([[-0.1985, -1.6639,  0.5235],
│                   [ 0.6780,  1.8817, -1.0716]])
└── 'x' --> <FastTreeValue 0x7f54b05f24f0>
    └── 'c' --> tensor([[ 0.5048, -1.0473, -0.0725, -1.5309],
                        [-0.3237,  0.9371,  0.7385, -0.7169],
                        [-0.5231,  2.0198, -1.1638, -0.4201]])

In [4]:
t.a

tensor([[-0.1985, -1.6639,  0.5235],
        [ 0.6780,  1.8817, -1.0716]])

In [5]:
%timeit t.a

61.5 ns ± 1.17 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f54b05f25b0>
├── 'a' --> tensor([[ 1.9353,  0.2036,  0.6295],
│                   [-1.1855, -0.7943,  1.6921]])
└── 'x' --> <FastTreeValue 0x7f54b05f24f0>
    └── 'c' --> tensor([[ 0.5048, -1.0473, -0.0725, -1.5309],
                        [-0.3237,  0.9371,  0.7385, -0.7169],
                        [-0.5231,  2.0198, -1.1638, -0.4201]])

In [7]:
%timeit t.a = new_value

69.5 ns ± 1.38 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.1985, -1.6639,  0.5235],
               [ 0.6780,  1.8817, -1.0716]]),
    x: Batch(
           c: tensor([[ 0.5048, -1.0473, -0.0725, -1.5309],
                      [-0.3237,  0.9371,  0.7385, -0.7169],
                      [-0.5231,  2.0198, -1.1638, -0.4201]]),
       ),
)

In [10]:
b.a

tensor([[-0.1985, -1.6639,  0.5235],
        [ 0.6780,  1.8817, -1.0716]])

In [11]:
%timeit b.a

57.8 ns ± 2.02 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.1893, -0.0825, -0.5325],
               [-0.7772, -0.3612,  2.6205]]),
    x: Batch(
           c: tensor([[ 0.5048, -1.0473, -0.0725, -1.5309],
                      [-0.3237,  0.9371,  0.7385, -0.7169],
                      [-0.5231,  2.0198, -1.1638, -0.4201]]),
       ),
)

In [13]:
%timeit b.a = new_value

469 ns ± 3.85 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

845 ns ± 35.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 549 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

156 µs ± 1.65 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

149 µs ± 2.79 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f54ac583460>
├── 'a' --> tensor([[[-0.1985, -1.6639,  0.5235],
│                    [ 0.6780,  1.8817, -1.0716]],
│           
│                   [[-0.1985, -1.6639,  0.5235],
│                    [ 0.6780,  1.8817, -1.0716]],
│           
│                   [[-0.1985, -1.6639,  0.5235],
│                    [ 0.6780,  1.8817, -1.0716]],
│           
│                   [[-0.1985, -1.6639,  0.5235],
│                    [ 0.6780,  1.8817, -1.0716]],
│           
│                   [[-0.1985, -1.6639,  0.5235],
│                    [ 0.6780,  1.8817, -1.0716]],
│           
│                   [[-0.1985, -1.6639,  0.5235],
│                    [ 0.6780,  1.8817, -1.0716]],
│           
│                   [[-0.1985, -1.6639,  0.5235],
│                    [ 0.6780,  1.8817, -1.0716]],
│           
│                   [[-0.1985, -1.6639,  0.5235],
│                    [ 0.6780,  1.8817, -1.0716]]])
└── 'x' --> <FastTreeValue 0x7f540128c880>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

34.3 µs ± 767 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f5401242f40>
├── 'a' --> tensor([[-0.1985, -1.6639,  0.5235],
│                   [ 0.6780,  1.8817, -1.0716],
│                   [-0.1985, -1.6639,  0.5235],
│                   [ 0.6780,  1.8817, -1.0716],
│                   [-0.1985, -1.6639,  0.5235],
│                   [ 0.6780,  1.8817, -1.0716],
│                   [-0.1985, -1.6639,  0.5235],
│                   [ 0.6780,  1.8817, -1.0716],
│                   [-0.1985, -1.6639,  0.5235],
│                   [ 0.6780,  1.8817, -1.0716],
│                   [-0.1985, -1.6639,  0.5235],
│                   [ 0.6780,  1.8817, -1.0716],
│                   [-0.1985, -1.6639,  0.5235],
│                   [ 0.6780,  1.8817, -1.0716],
│                   [-0.1985, -1.6639,  0.5235],
│                   [ 0.6780,  1.8817, -1.0716]])
└── 'x' --> <FastTreeValue 0x7f5401297a60>
    └── 'c' --> tensor([[ 0.5048, -1.0473, -0.0725, -1.5309],
                        [-0.3237,  0.9371,  0.7385, -0.7169],
                 

In [23]:
%timeit t_cat(trees)

30.5 µs ± 1.46 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.6 µs ± 2.61 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.1985, -1.6639,  0.5235],
                [ 0.6780,  1.8817, -1.0716]],
       
               [[-0.1985, -1.6639,  0.5235],
                [ 0.6780,  1.8817, -1.0716]],
       
               [[-0.1985, -1.6639,  0.5235],
                [ 0.6780,  1.8817, -1.0716]],
       
               [[-0.1985, -1.6639,  0.5235],
                [ 0.6780,  1.8817, -1.0716]],
       
               [[-0.1985, -1.6639,  0.5235],
                [ 0.6780,  1.8817, -1.0716]],
       
               [[-0.1985, -1.6639,  0.5235],
                [ 0.6780,  1.8817, -1.0716]],
       
               [[-0.1985, -1.6639,  0.5235],
                [ 0.6780,  1.8817, -1.0716]],
       
               [[-0.1985, -1.6639,  0.5235],
                [ 0.6780,  1.8817, -1.0716]]]),
    x: Batch(
           c: tensor([[[ 0.5048, -1.0473, -0.0725, -1.5309],
                       [-0.3237,  0.9371,  0.7385, -0.7169],
                       [-0.5231,  2.0198, -1.1638, -0.4201]],
         

In [26]:
%timeit Batch.stack(batches)

79.2 µs ± 3.17 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.1985, -1.6639,  0.5235],
               [ 0.6780,  1.8817, -1.0716],
               [-0.1985, -1.6639,  0.5235],
               [ 0.6780,  1.8817, -1.0716],
               [-0.1985, -1.6639,  0.5235],
               [ 0.6780,  1.8817, -1.0716],
               [-0.1985, -1.6639,  0.5235],
               [ 0.6780,  1.8817, -1.0716],
               [-0.1985, -1.6639,  0.5235],
               [ 0.6780,  1.8817, -1.0716],
               [-0.1985, -1.6639,  0.5235],
               [ 0.6780,  1.8817, -1.0716],
               [-0.1985, -1.6639,  0.5235],
               [ 0.6780,  1.8817, -1.0716],
               [-0.1985, -1.6639,  0.5235],
               [ 0.6780,  1.8817, -1.0716]]),
    x: Batch(
           c: tensor([[ 0.5048, -1.0473, -0.0725, -1.5309],
                      [-0.3237,  0.9371,  0.7385, -0.7169],
                      [-0.5231,  2.0198, -1.1638, -0.4201],
                      [ 0.5048, -1.0473, -0.0725, -1.5309],
                      [-0.3237,  

In [28]:
%timeit Batch.cat(batches)

148 µs ± 4.07 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

342 µs ± 8.92 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
